In [ ]:
!pip install python-Levenshtein

# Laboratorio #3: Expansión de consultas

La expansión de consultas es una técnica importante en la recuperación de información y en los motores de búsqueda por varias razones:
1. Mejora de la precisión y recobrado:
   - Al incluir términos relacionados y sinónimos se puede mejorar las métricas de los resultados de búsqueda. Esto ayuda a capturar documentos relevantes que podrían haber sido ignorados con la consulta original.

2. Reducción de ambigüedad:
   - Las consultas iniciales a menudo son ambiguas o demasiado breves. Expandir la consulta puede aclarar la intención del usuario y reducir la ambigüedad, mejorando la relevancia de los resultados.

3. Manejo de sinónimos y variaciones lingüísticas:
   - Asegura que se encuentren documentos relevantes que utilizan diferentes términos para referirse al mismo concepto.

4. Mejora la experiencia de usuario:
   - Proporciona una mejor experiencia de usuario al devolver resultados más completos y relevantes, lo que puede aumentar la satisfacción y la confianza en el sistema de búsqueda.

5. Optimización para motores de búsqueda:
   - En el contexto del SEO (Search Engine Optimization), expandir consultas con palabras clave relacionadas puede mejorar la visibilidad de los contenidos en los motores de búsqueda, atrayendo más tráfico y mejorando el posicionamiento.

Existen distintos tipos de métodos y técnicas para expandir la consulta efectuada por un usuario. Los más comunes son:

1. Reformulación de la consula
  - Sinónimos y términos relacionados: Uso de tesauros y/o diccionarios para encontrar sinónimos y términos relacionados que puedan añadirse a la consulta.
  - Expansión manual: Pedirle a los usuarios que proporcionen términos adicionales o frases que consideren relevantes, a partir de la consulta definida.

2. Retroalimentación
  - Retroalimentación Relevante: Utiliza la retroalimentación de relevancia en la que los usuarios marcan documentos relevantes. Luego, ajusta la consulta para incluir términos encontrados en estos documentos.
  - Pseudo-relevancia: Considera los primeros resultados de la búsqueda como relevantes y usa estos documentos para expandir la consulta.

3. Modelos de Lenguaje y Procesamiento del Lenguaje Natural 
  - Modelos de lenguaje: Utiliza modelos BERT o GPT para sugerir términos relacionados y expandir la consulta.
  - Word embeddings: Usa técnicas como Word2Vec o GloVe para encontrar términos semánticamente similares y añadirlos a la consulta.

4. Expansión basada en la búsqueda
  - Exploración de términos: Extraer términos importantes de los N primeros documentos recuperados inicialmente y usa estos términos para expandir la consulta.
  - Consulta dividida: Divide la consulta en subconsultas más pequeñas y combina los resultados.


Durante el laboratorio, se implementarán distintos métodos para expandir una consulta.

In [6]:
# Importando librerías necesarias

# Fuente del corpus
import ir_datasets

# Útil para el trabajo con arreglo numéricos
import numpy as np

# Facilita el trabajo con los términos indexados del corpus
from gensim.corpora.dictionary import Dictionary

# Interfaz para el trabajo con Wordnet
from nltk.corpus import wordnet as wn

# Útil para calcular la distancia de Levenshtein entre 2 palabras
from Levenshtein import distance as distance_Levenshtein

# Funciones útiles auxiliares
from teacher_help import tokenize

### **Ejercicio #1**: Implemente la expasión de consultas usando la matriz de co-ocurrencia.

a) Explique cómo se construye la matriz co-ocurrencia.

b) Implemente una función que dado un corpus (Cranfield) tokenizado, devuelva:
- la matriz de co-ocurrencia normalizada (valores de 0 a 1),
- un diccionario que permita dado un token, obtener un número único asociado al token, el cual es a su vez único dentro del vocabulario, y
- un diccionario que permita dado un número, obtener el token asociado dentro del vocabulario.

Considere utilizar solo los tokens "más" importantes para el análisis.

c) Implemente una función que dada la matriz de co-ocurrencia y un token, devuelva aquellos tokens similares a partir del corpus.

In [ ]:
# raise NotImplemented('Ejercicio 1.b - Cargando el corpus')

corpus = ir_datasets.load("cranfield")
tokenized_corpus = [tokenize(doc[2], ['ADJ', 'NOUN', 'PROPN', 'VERB']) for doc in dataset.docs_iter()]

In [ ]:
def build_cooccurrence_matrix(tokenized_corpus, window_size=float('inf')):
    """
    Build the co-occurrence matrix from a corpus
    
    Args:
    - tokenized_corpus : [[str]]
        Tokenized corpus, each element represents a document.
    - window_size : int
        Window size to consider that 2 tokens occur. By default, the value `inf` is defined, which represents the same document as the window size.
    
    Returns:
    - (A, B, C) where:
        A np.ndarray represents the co-occurrence matrix (symmetric).
        B {str: int} represents the encoding of each token in the vocabulary.
        C {int: str} represents the token of each code.
        
    """
    # raise NotImplementedError("Ejercicio 1.b")
    
    vocabulary = Dictionary(tokenized_corpus)
    vocab_size = len(vocabulary)
    
    # Codifico cada documento, para no tener que hacerlo por cada token dentro del doble for 
    encode_corpus = [vocabulary.doc2idx(doc) for doc in tokenized_corpus]
    
    cooccurrence_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for encode_doc in encode_corpus:
        for i, encode_word in enumerate(encode_doc):
            start = max(i - window_size, 0)
            end = min(i + window_size + 1, len(encode_doc))
            
            for j in encode_doc[start:end]:
                if j != i:
                    # Matriz simétrica
                    cooccurrence_matrix[i][j] += 1
                    cooccurrence_matrix[j][i] += 1
    
    normalized_matrix = (cooccurrence_matrix - cooccurrence_matrix.min()) / (cooccurrence_matrix.max() - cooccurrence_matrix.min())
    return normalized_matrix, vocabulary.token2id, {id: word for word, id in vocabulary.token2id.items()}

cooccurrence_matrix, token2id, id2token = build_cooccurrence_matrix(tokenized_corpus)

In [ ]:
def get_similar_terms(token, cooccurrence_matrix, token2id, id2token, threshold=0.7, top_n=5):
    """
    Returns terms close to a token
    
    Args:
    - token : str
        Token.
    - cooccurrence_matrix : np.ndarray
        Co-occurrence matrix, symmetric. The indices match the vocabulary encoding defined in `token2id`.
    - token2id : {str: int}
        Dictionary where the keys are the vocabulary tokens and the value is a unique numerical code.
    - id2token : {int: str}
        Dictionary where the keys are the code (unique numeric) and the value is a unique token within the vocabulary.
    - threshold : float
        Numeric value between 0 and 1, represents the minimum value that the token must have to be returned.
    - top_n : int
        Number of words to recover.
        
    Return:
    - [str]
    
    """
    # raise NotImplementedError("Ejercicio 1.c")
    
    if not token in token2id:
        return []
    
    token_id = token2id[token]
    words = [(value, id2token[pos]) for pos, value in enumerate(cooccurrence_matrix[token_id]) if pos != token_id and value >= threshold]
    words = sorted(words, reverse=True)
    return [t for _, t in words[:top_n]]


In [ ]:
token = 'aerodynamic'

# Consultar la ayuda de la función get_similar_terms/6 para ver el significado de los siguientes parámetros
threshold = 0.3
top_n = 10

similar_terms = get_similar_terms(token, cooccurrence_matrix, token2id, id2token, threshold, top_n)

print(f'Los tokens más similares a `{token}` utilizando matriz de co-ocurrencia son: \n{similar_terms}')

---

En la práctica, si por ejemplo se tiene la consulta 

`Impacto del cambio climático en la biodiversidad marina`

Luego de expandir la consulta, se obtiene:

`cambio climático biodiversidad marina impacto efecto consecuencia política economía publicado:2015..2024`

Es cierto que al sistema "solo" le interesa los tokens, sin importar el orden pero de esta manera se obvian algunos aspectos, como:
- No se le da mayor relevancia a los tokens de la consulta original, teniendo todos la misma relevancia. Esto puede no ser cierto cuando se expande la consulta con términos muy generales o muy específicos, sacando al usuario de la zona de búsqueda esperada.
- Puede existir una sobrecarga de información en la consulta expandida, conllevando a obtener resultados irrelevantes.
- Aumenta la ambigüedad de la consulta.
- No es trivial convertir la consulta en subconsultas.

Por tanto, es conveniente definir una estructura donde se establezcan reglas, esto se conoce como **estructuración de la consulta**.

Esta técnica se define a través de 3 operadores:
1. Booleanos: Utiliza AND, OR y NOT para combinar términos.
2. De agrupación: Utiliza paréntesis para agrupar términos.
3. Para filtros específicos: Añade filtros a partir de un formato predefinido en el sistema.
   
Por tanto, la consulta 

`impacto del cambio climático en la biodiversidad marina`

puede estructurarse de la siguiente manera:

`("cambio climático" AND "biodiversidad marina") AND ("impacto" OR "efecto" OR "consecuencia") NOT ("política" OR "economía") AND (publicado:2015..2024)` 

donde:
- `("cambio climático" AND "biodiversidad marina")`: Asegura que ambos términos "cambio climático" y "biodiversidad marina" aparezcan juntos.
- `AND ("impacto" OR "efecto" OR "consecuencia")`: Añade términos relacionados con el impacto, aumentando la probabilidad de encontrar artículos relevantes. Al utilizar OR, ampliamos la búsqueda para incluir cualquiera de estos términos.
- `NOT ("política" OR "economía")`: Excluye resultados relacionados con política o economía que pueden no ser relevantes para el enfoque científico o biológico deseado.
- `AND (publicado:2015..2024)`: Añade un filtro de rango de fechas para obtener artículos publicados entre 2015 y 2024, asegurando que la información sea reciente.

### **Ejercicio #2:** Implemente una función que estructure una consulta.

Para la realización de esta función, considere:
- No trabajará con el operador NOT.
- Todos los tokens identificados en la consulta original, tienen que aparecer en la consulta expandida.
- Los tokens similares a añadir, será provisto de una función externa. Puede reutilizar la recién jecho con la matriz de co-ocurrencia.

In [ ]:
def structure_query(tokenized_query, get_similar_tokens_function):
    """
    Structure a query

    Args:
    - tokenized_query : [str]
        Tokenized query.
    - get_similar_tokens_function : function
        Function to obtain the tokens most similar to another. It is assumed to have cardinality 1.
        
    Return:
    - str
            
    """
    # raise NotImplementedError('Ejercicio 2')

    groups = [] 

    for token in tokenized_query:
        similars = get_similar_tokens_function(token) 
        if similars:
            group = [token] + similars
            groups.append(f"({' OR '.join(similars)})")
        else:
            groups.append(token)

    return ' AND '.join(groups)

In [ ]:
query = 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft.'
tokenized_query = tokenize(query, ['ADJ', 'NOUN', 'PROPN', 'VERB'])
                           
# Imprimir la consulta original y la consulta estructurada
print(f"Consulta original: \n{query}")
print(f"\nConsulta expandida y estructurada: \n{structure_query(tokenized_query, lambda token: get_similar_terms(token, cooccurrence_matrix, token2id, id2token, threshold, top_n))}")

---

La **distancia de Levenshtein**, también conocida como distancia de edición, es una métrica utilizada para medir la diferencia entre dos cadenas de texto. Específicamente, cuantifica el número mínimo de operaciones necesarias para transformar una cadena en otra. Las operaciones permitidas son:
- Inserción: Añadir un carácter.
- Eliminación: Quitar un carácter.
- Sustitución: Cambiar un carácter por otro.

Esta distancia es muy usada para:
- sugerir correcciones ortográficas basadas en la similitud con palabras correctas. (Corrección Ortográfica)
- ayudar a la identificación de patrones y coincidencias en el procesamiento de texto. (Reconocimiento de Patrones)
- comparar secuencias de ADN. (Comparación de Cadenas de ADN)
- la búsqueda difusa y la coincidencia aproximada de cadenas. (Procesamiento del Lenguaje Natural)

Por su parte, es muy usual encontrarse con consultas donde sus términos presentan errores ortográficos. A consecuencia de esto, el sistema debe de ser capaz de detectar los errores ortográficos e identificar el token o la palabra por la cual debe de sustituirse.

### Ejercicio #3: Construya una función que dada una consulta tokenizada, se arreglen los errores ortográficos presentes en la consulta.

In [7]:
def fix_spelling_errors(tokenized_query, vocabulary, threshold):
    """
    Analyze a query and fix spelling errors present

    Args:
    - tokenized_query : [str]
        Tokenized query.
    - vocabulary: [str]
        Vocabulary.
    - threshold : int
        Maximum number of changes to consider a word close to another with a spelling error. The value is assumed to be non-negative.

    Return:
    - [str]
    
    """
    def correct_word_func(word, vocabulary, threshold):
        best_word = None
        best_distance = float('inf')

        for word_v in vocabulary:
            distance = distance_Levenshtein(word, word_v)
            if distance < best_distance:
                best_distance = distance
                best_word = word_v
                
        return best_word if best_distance <= threshold else None
    
    corrected_words = []
    for word in tokenized_query:
        correct_word = correct_word_func(word, vocabulary, threshold)
        if correct_word:
            corrected_words.append(correct_word)
    return corrected_words

In [10]:
query = 'whatt simmilarity laws msut be obayed wen consructing aeroelstic moddels of heateed high speeed aircaft.'
tokenized_query = tokenize(query, ['ADJ', 'NOUN', 'PROPN', 'VERB'])

vocabulary = None
threshold = 100

fix_spelling_errors(tokenized_query, vocabulary, threshold)

['kitten', 'sitting', 'apple', 'banana']

---

**WordNet** es una base de datos léxica del idioma inglés desarrollada por el Cognitive Science Laboratory de la Universidad de Princeton. Está organizada en grupos de términos conocidos como synsets (conjuntos de términos con relaciones), que representan conceptos específicos. Cada synset está vinculado a otros synsets a través de diversas relaciones semánticas y léxicas, como la hiponimia (relación entre términos más específicos y más generales) y la sinonimia (relación de sinónimos).

WordNet se utiliza ampliamente en el procesamiento del lenguaje natural, la lingüística computacional y la inteligencia artificial debido a sus capacidades para:
- Desambiguación de palabras.
- Expansión de consultas.
- Análisis semántico.
- Generación de texto y traducción automática.
- Desarrollo de aplicaciones de lenguaje.

### **Ejercicio 4**: Utilizando WordNet, implemente una función para expandir una consulta. 

Esta función debe de hacer uso de los synsets, hipónimos e hiperónimos de cada término. Recuerde que:
- Hipónimos: Son términos más específicos que la palabra dada (subclases).
- Hiperónimos: Son términos más generales que la palabra dada (superclases).

###### Ayuda: Considere usar las funciones: synsets/1, hyponyms/1, hypernyms/1 y lemmas/0

In [106]:
def get_similar_terms_wordnet(token):
    """
    Returns the synsets, hypernyms, and hyponyms of a token

    Args:
    - token : str
        Token.
        
    Return: 
    - dict
        
    """
    # raise NotImplementedError('Ejercicio 4')
    
    synset_set = set()
    hyponyms_set = set()
    hypernyms_set = set()
    
    for synset in wn.synsets(palabra):
        for lemma in synset.lemmas():
            synset_set.add(lemma.name())
        
        for hyponym in synset.hyponyms():
            for lemma in hyponym.lemmas():
                hyponyms_set.add(lemma.name())
                
        for hypernym in synset.hypernyms():
            for lemma in hypernym.lemmas():
                hypernyms_set.add(lemma.name())
    
    return {
        'synsets': list(synset_set),
        'hyponyms': list(hyponyms_set),
        'hypernyms': list(hypernyms_set)
    }

In [ ]:
token = 'data'
similars = get_similar_terms_wordnet(token)

print(f"Similares a '{token}':")
for sinonimo in similars['synsets']:
    print(f"- {sinonimo}")

print(f"\nHipónimos de '{token}':")
for hiponimo in similars['hyponyms']:
    print(f"- {hiponimo}")

print(f"\nHiperónimos de '{token}':")
for hiperónimo in similars['hypernyms']:
    print(f"- {hiperónimo}")